In [74]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import scipy.linalg
import scipy.stats
import copy
import shapely.wkt 
import itertools
import multiprocessing as mp
from descartes import PolygonPatch

from functools import partial
from sklearn.neighbors import BallTree
from shapely.geometry import Point,LineString
from tqdm import tqdm
from tqdm.notebook import trange,tqdm_notebook

# Import data

In [126]:
# Import camera metadata
camera_buffers_filename = "../data/raw/camera_buffer_zones_separated.geojson"
# camera_buffers = gpd.read_file(camera_buffers_filename)
camera_buffers_dissolved_filename = "../data/raw/camera_buffer_zones.geojson"
camera_buffers_dissolved = gpd.read_file(camera_buffers_dissolved_filename)

# Import road metadata
london_roads_filename = "../data/raw/os_highway_tfl_road.geojson"
# "../data/raw/os_highway_tfl_road.geojson"
# "../data/raw/arup_network_links_london2.geojson"

london_roads = gpd.read_file(london_roads_filename)
london_roads_copy = copy.deepcopy(london_roads)

arup_dataset = False

# Clean data

In [55]:
# Keep only working cameras
# camera_buffers = camera_buffers[camera_buffers.flag == 0]
# Keep only necessary columns
camera_buffers = camera_buffers[['id', 'camera_id', 'notes','borough_name', 'borough_gss_code','flag','geometry']]

In [132]:
if arup_dataset:
    # Remove unnecessary data
    london_roads = london_roads.drop(columns=['s2_from','s2_to','ids','u','v','key'])
    london_roads = london_roads.rename(columns={"id":"road_id"})
    london_roads = london_roads[london_roads.modes.str.contains("car")]
    # Export roads only
    # london_roads.to_file("../data/raw/arup_road_links_london2.geojson", driver='GeoJSON', crs='EPSG:4326')
else:
    # Remove unnecessary data
#     london_roads = london_roads.drop(columns=['s2_from','s2_to','ids','u','v','key'])
    london_roads = london_roads.rename(columns={"toid":"road_id"})

# Plot dissolved camera buffer multipolygon

In [133]:
# BLUE = '#6699cc'
# fig = plt.figure() 
# ax = fig.gca() 
# ax.add_patch(PolygonPatch(camerdda_buffers_dissolved, fc=BLUE, ec=BLUE, alpha=0.5, zorder=2 ))
# ax.axis('scaled')
# plt.show()

# Find road within buffer

In [135]:
sample_road_df = london_roads[london_roads.road_id == '80402']
# 85025
# 80402

In [136]:
def find_overlapping_roads_per_buffer(roads,buffers):
    # Loop each road 
    n_buffers = np.unique(buffers.camera_id).shape[0]
    roads_per_buffer = dict(zip(np.unique(buffers.camera_id), [[] for i in range(n_buffers)]))
    for rindex, road in tqdm_notebook(roads.iterrows()):
        for bindex, buffer in tqdm_notebook(buffers.iterrows(),leave=False):
            if road.geometry.intersects(buffer.geometry):
                roads_per_buffer[buffer.camera_id].append(road.road_id)
    return roads_per_buffer
def find_overlapping_roads(roads,dissolved_buffer):
    # Loop each road 
    buffer_roads = []
    for rindex, road in tqdm_notebook(roads.iterrows(),total=roads.shape[0]):
        if road.geometry.intersects(dissolved_buffer.geometry[0]):
            buffer_roads.append(road.road_id)
    return buffer_roads

In [137]:
# roads_per_camera_buffer = find_overlapping_roads_per_buffer(sample_road_df,camera_buffers)
camera_buffer_road_ids = find_overlapping_roads(london_roads,camera_buffers_dissolved)

  0%|          | 0/339214 [00:00<?, ?it/s]

In [138]:
print(f'Roads reduced from {london_roads.shape[0]} to {len(camera_buffer_road_ids)}')

Roads reduced from 339214 to 15114


In [139]:
if arup_dataset:
    # Get london roads intersecting dissolved camera buffer
    london_roads_subset = london_roads[london_roads.road_id.isin(camera_buffer_road_ids)]
else:
    london_roads_subset = london_roads[london_roads.road_id.isin(camera_buffer_road_ids)]

## Export data

In [140]:
if arup_dataset:
    london_roads_subset.to_file('../data/output/gis/arup_roads2_within_camera_buffers.geojson', driver='GeoJSON', crs='EPSG:4326')
else:
    london_roads_subset.to_file('../data/output/gis/tfl_roads_within_camera_buffers.geojson', driver='GeoJSON', crs='EPSG:4326')